## Problem 1 — Binary Words and Operations

This problem asks us to implement several low-level bitwise functions used in SHA-256.  
They are defined in the **Secure Hash Standard (FIPS 180-4)**.  
All operations must be performed on **32-bit unsigned integers**, so I use `numpy.uint32`.

The functions implemented are:

- Parity(x, y, z)
- Ch(x, y, z)
- Maj(x, y, z)
- Σ0(x), Σ1(x)
- σ0(x), σ1(x)

Each function is introduced in its own section below.


In [46]:
import numpy as np  # For 32-bit unsigned integers (np.uint32)


def rotr(x, n):
    """
    Rotate a 32-bit value x to the right by n bits.
    Used in almost all SHA-256 logical functions.
    """
    x = np.uint32(x)
    n = n % 32
    return np.uint32((x >> n) | (x << (32 - n)))


def shr(x, n):
    """
    Logical right shift of x by n bits.
    Unlike rotation, bits shifted off the right are discarded.
    """
    x = np.uint32(x)
    return np.uint32(x >> n)


## Problem 1A: Parity(x, y, z)

**Goal:** Write a function that returns the XOR of three 32-bit numbers.

From the standard (FIPS 180-4):  
**Parity(x, y, z) = x ⊕ y ⊕ z**

This function is very simple — XOR all three values.  
We ensure everything is cast to `np.uint32` so the result stays 32-bit.


In [47]:
def Parity(x, y, z):

    """Return Parity(x, y, z) = x XOR y XOR z operating on 32-bit words.



    - Inputs may be scalars or numpy arrays; they are cast to np.uint32.

    - The result preserves shape and dtype (np.uint32).

    """

    return u32(u32(x) ^ u32(y) ^ u32(z))

def Parity(x, y, z):
    """
    XOR all three 32-bit words together.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return np.uint32(x ^ y ^ z)


## Problem 1B: Ch(x, y, z)

**Goal:** Implement the SHA-256 “choose” function.

From the standard:  
**Ch(x, y, z) = (x AND y) XOR (~x AND z)**

Interpretation:  
- If a bit of **x** is 1 → choose bit from **y**  
- If a bit of **x** is 0 → choose bit from **z**  


In [48]:
def Ch(x, y, z):
    """
    SHA-256 choice function:
    For each bit, choose from y if x's bit is 1, else from z.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return np.uint32((x & y) ^ (~x & z))


## Problem 1C: Maj(x, y, z)

**Goal:** Implement the SHA-256 majority function.

From the standard:  
**Maj(x, y, z) = (x AND y) XOR (x AND z) XOR (y AND z)**

Interpretation:  
Bit becomes **1** if *at least two* of x, y, z have a 1 in that position.


In [49]:
def Maj(x, y, z):
    """
    SHA-256 majority function.
    Bit is 1 if two or more of (x, y, z) have bit 1.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return np.uint32((x & y) ^ (x & z) ^ (y & z))


## Problem 1D: Σ0(x) and Σ1(x)

**Goal:** Implement the two uppercase sigma functions from the standard.  

These use *rotations*, not shifts.

Definitions (FIPS 180-4):

- **Σ0(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)**
- **Σ1(x) = ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)**

These are used in the SHA-256 compression function.


In [50]:
def Sigma0(x):
    """
    Σ0(x) = ROTR^2(x) ^ ROTR^13(x) ^ ROTR^22(x)
    """
    x = np.uint32(x)
    return np.uint32(rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))


def Sigma1(x):
    """
    Σ1(x) = ROTR^6(x) ^ ROTR^11(x) ^ ROTR^25(x)
    """
    x = np.uint32(x)
    return np.uint32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))


## Problem 1E: σ0(x) and σ1(x)

**Goal:** Implement the lowercase sigma functions used in the message schedule.  

These mix **rotation** and **logical shift**.

Definitions (FIPS 180-4):

- **σ0(x) = ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x)**  
- **σ1(x) = ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x)**  

These are used when generating the message schedule array `W[t]`.

In [51]:
def sigma0(x):
    """
    σ0(x) = ROTR^7(x) ^ ROTR^18(x) ^ SHR^3(x)
    """
    x = np.uint32(x)
    return np.uint32(rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3))


def sigma1(x):
    """
    σ1(x) = ROTR^17(x) ^ ROTR^19(x) ^ SHR^10(x)
    """
    x = np.uint32(x)
    return np.uint32(rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10))


## Problem 1F: Testing all functions

To verify correctness, I test the functions using small example numbers.  
I print the results in hexadecimal (easier to compare with the standard).

In [52]:
a = np.uint32(0x0F0F0F0F)
b = np.uint32(0x33333333)
c = np.uint32(0xAAAAAAAA)

print("Parity =", hex(a ^ b ^ c))
print("Ch     =", hex(Ch(a, b, c)))
print("Maj    =", hex(Maj(a, b, c)))

x = np.uint32(0x12345678)

print("\nSigma0 =", hex(Sigma0(x)))
print("Sigma1 =", hex(Sigma1(x)))
print("sigma0 =", hex(sigma0(x)))
print("sigma1 =", hex(sigma1(x)))


Parity = 0x96969696
Ch     = 0xa3a3a3a3
Maj    = 0x2b2b2b2b

Sigma0 = 0x66146474
Sigma1 = 0x3561abda
sigma0 = 0xe7fce6ee
sigma1 = 0xa1f78649


## Problem 2: Fractional Parts of Cube Roots

**Goal:** Recreate the SHA-256 constants `K[0..63]` from the Secure Hash Standard (FIPS 180-4).

According to the standard, each constant is:

> "the first 32 bits of the fractional parts of the cube roots of the first 64 prime numbers."

So the plan is:

1. Write a function `primes(n)` that returns the first `n` prime numbers.
2. Use NumPy (`np.cbrt`) to calculate the cube root of each prime.  
   See: [NumPy cbrt docs](https://numpy.org/doc/stable/reference/generated/numpy.cbrt.html)
3. Extract the **fractional part** of each cube root.
4. Multiply the fractional part by \( 2^{32} \), take the integer part, and store it as a 32-bit value (`np.uint32`).
5. Display the result in **hexadecimal** and compare manually with the constants listed in FIPS 180-4.


In [53]:
import numpy as np  # For 32-bit unsigned integers (np.uint32)

def primes(n):
    """
    Return a list of the first n prime numbers.

    I use a simple trial division approach:
    - Start at 2.
    - For each candidate, test divisibility by earlier primes.
    - Only test up to the square root of the candidate.
    """
    # If n is zero or negative, just return an empty list.
    if n <= 0:
        return []

    result = []      # list to store primes
    candidate = 2    # first number to test for primality

    # Keep going until we have n primes.
    while len(result) < n:
        is_prime = True  # assume candidate is prime until shown otherwise

        # Check divisibility by previously found primes.
        for p in result:
            # If p^2 is greater than candidate, we can stop checking.
            if p * p > candidate:
                break
            # If candidate is divisible by p, it is not prime.
            if candidate % p == 0:
                is_prime = False
                break

        # If no divisors were found, we have a new prime.
        if is_prime:
            result.append(candidate)

        # Move on to the next integer.
        candidate += 1

    return result


### Problem 2A: Testing the `primes(n)` function

Before using `primes(n)` for the main task, I want to quickly check that it returns
the correct small primes.

Known values:
- First 5 primes: 2, 3, 5, 7, 11
- First 10 primes: 2, 3, 5, 7, 11, 13, 17, 19, 23, 29


In [54]:
# Quick sanity checks for primes(n).

first_5 = primes(5)
first_10 = primes(10)

print("First 5 primes: ", first_5)
print("First 10 primes:", first_10)


First 5 primes:  [2, 3, 5, 7, 11]
First 10 primes: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]


### Problem 2B: Cube roots and fractional parts

Next I need to convert each prime into a 32-bit constant as follows:

1. Compute the cube root of the prime using `np.cbrt(p)`.
2. Extract the fractional part:

   \[
   \text{frac} = \text{cuberoot}(p) - \lfloor \text{cuberoot}(p) \rfloor
   \]

3. Multiply the fractional part by \( 2^{32} \).
4. Take the integer part (floor) and store it as a 32-bit unsigned integer (`np.uint32`).

This should match the way SHA-256 defines its `K` constants.


In [55]:
def cube_root_fraction_to_uint32(p):
    """
    Given a prime p, compute the 32-bit word defined as:
        floor( fractional_part(cuberoot(p)) * 2**32 )

    The result is returned as a NumPy uint32, to match the 32-bit word
    behaviour in the SHA-256 standard.
    """
    # Convert p to a float64 so np.cbrt can operate on it.
    x = np.float64(p)

    # Compute the cube root using NumPy.
    cbrt = np.cbrt(x)

    # Fractional part: total value minus its floor.
    frac = cbrt - np.floor(cbrt)

    # Scale the fractional part to 32 bits.
    scaled = frac * (2**32)

    # Take the integer part.
    integer_bits = int(scaled)

    # Cast to 32-bit unsigned integer for SHA-256.
    return np.uint32(integer_bits)


In [56]:
## Problem 3: Padding
"""I'll do this section later. Leaving this here to remember the order."""

"I'll do this section later. Leaving this here to remember the order."

In [57]:
## Problem 4: Hashes
"""I'll do this section later. Leaving this here to remember the order."""

"I'll do this section later. Leaving this here to remember the order."

In [58]:
## Problem 5: Passwords 
"""I'll do this section later. Leaving this here to remember the order."""

"I'll do this section later. Leaving this here to remember the order."